This notebook is used to clean and prepare the genre label data for analysis.

## Outline of Cleaning:

- [x] remove artists for which 'retrieved' value is 'none'
- [x] remove the url prefix from the retrieved artist names 
- [x] replace ' ' in the artist column with '_'
- [x] remove the '(singer)', '(rapper)', '(musician)' designation from the 'retrieved' column
- [x] remove the artists for which the retrieved-artist != searched-artist. 
    - inspect mismatches to look for typos and different versions
- [x] convert genre column values into lists of strings
- [x] remove old columns
- [x] extract unique genres as a list
- [ ] select 1% sample to verify gender

In [1]:
import numpy as np
np.random.seed(23)
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import re

In [2]:
%ls -lt /Users/Daniel/Code/Genre/data/genre_lists/

total 20328
-rw-r--r--  1 Daniel  staff   184182 Apr 24 16:44 wiki-women_wiki_lists.csv
-rw-r--r--@ 1 Daniel  staff  1901830 Apr 23 13:41 wiki-kaggle_genres_gender.csv
-rw-r--r--@ 1 Daniel  staff  2010856 Apr 23 13:41 wiki-kaggle_genres_rough_gender.csv
-rw-r--r--@ 1 Daniel  staff   708936 Apr 22 15:30 wiki-kaggle_genres_rough.csv
-rw-r--r--  1 Daniel  staff   895886 Apr 17 10:27 kaggle_genres-reduced.csv
-rwxr-xr-x@ 1 Daniel  staff    22397 Apr 15 13:24 wiki_bands_women.csv*
-rwxr-xr-x@ 1 Daniel  staff    37575 Apr 15 13:24 wiki_country_women.csv*
-rwxr-xr-x@ 1 Daniel  staff    79119 Apr 15 13:24 wiki_rock_women.csv*
-rw-r--r--@ 1 Daniel  staff  1949982 Apr 15 11:34 kaggle_genres_rough.csv
-rwxr-xr-x@ 1 Daniel  staff    15444 Apr 11 12:41 wiki_bands_women-cleaned.csv*
-rwxr-xr-x@ 1 Daniel  staff    24278 Apr 11 12:41 wiki_country_women-cleaned.csv*
-rwxr-xr-x@ 1 Daniel  staff    53345 Apr 11 12:41 wiki_rock_women-cleaned.csv*
-rwxr-xr-x@ 1 Daniel  staff     8351 Apr  8 13:19 women_in_

### Data Sets

The file singers_gender.csv is from Kaggle and lists music artists and their gender. This is our starting point. It is augmented using the lists of women artists. Genre and network info will be generated by scraping databases. For now we are focusing in Wikipedia.

In [3]:
# kaggle_data = pd.read_csv('singers_gender.csv', encoding = 'latin-1')

In [4]:
# kaggle_data.shape

### Load the data to be cleaned:

Current: wiki-kaggle_genres_rough.csv

- This will be replaced by the fully scraped set
- The full set needs to be cleaned

Add in a converter that splits the genre list on commas:
https://stackoverflow.com/questions/32742976/how-to-read-a-column-of-csv-as-dtype-list-using-pandas

In [5]:
data = pd.read_csv('../../data/genre_lists/wiki-kaggle_genres_rough_gender.csv', header = 0)
data.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [6]:
data.head()

,artist,gender,retrieved,genre
0,Billy Joe Royal,male,https://en.wikipedia.org/wiki/Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus..."
1,Inés Gaviria,female,https://en.wikipedia.org/wiki/Inés_Gaviria,"['Latin pop', 'Pop rock']"
2,Tha Realest,male,https://en.wikipedia.org/wiki/Tha_Realest,['Hip hop music']
3,Mr. Lif,male,https://en.wikipedia.org/wiki/Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli..."
4,Clarice Mayne,female,https://en.wikipedia.org/wiki/Clarice_Mayne,"['Music hall', 'Variety theatre']"


In [7]:
data.shape

(23177, 4)

In [8]:
data.isnull().sum()

artist       0
gender       0
retrieved    0
genre        0
dtype: int64

For how many artists is the scraped genre 'none':

In [9]:
(data.genre == 'none').sum()

7704

For how many artists is the 'retrieved' value 'none':

In [10]:
(data.retrieved == 'none').sum()

7704

Take a glance at artist and retrieved values to determine necessary cleaning:

In [17]:
rints = np.random.randint(0,data.shape[0],15) # generate 15 random numbers from 0 to k-1, with k = # of rows

for n in rints:
    print('artist: {}        retrieved: {}'.format(data.artist.iloc[n], data.retrieved.iloc[n]))

artist: Alexandra Deshorties        retrieved: none
artist: Mark Vieha        retrieved: none
artist: Aidan Knight        retrieved: https://en.wikipedia.org/wiki/Aidan_Knight
artist: Butterscotch        retrieved: https://en.wikipedia.org/wiki/Butterscotch_(singer)
artist: Jani Lauzon        retrieved: none
artist: Tom Brosseau        retrieved: https://en.wikipedia.org/wiki/Tom_Brosseau
artist: Chris Medina        retrieved: https://en.wikipedia.org/wiki/Chris_Medina
artist: Lobo        retrieved: https://en.wikipedia.org/wiki/Lobo_(musician)
artist: Nick Loren        retrieved: none
artist: Álex Lora        retrieved: none
artist: D'Angelo        retrieved: https://en.wikipedia.org/wiki/D'Angelo
artist: Jinder        retrieved: none
artist: Steve Hart        retrieved: https://en.wikipedia.org/wiki/Steve_Hart_(singer)
artist: Patrick Bourque        retrieved: https://en.wikipedia.org/wiki/Patrick_Bourque
artist: Natalia Yanchak        retrieved: none


Notes on Retrieved:

- Underscore is used to separate parts of the name
- '.' are allowed in names 
- '(singer)' and '(musician)' are sometimes included and need to be stripped (probably to distinguish from othe people in wikipedia)
- double quotes are allowed in names
- hyphens appear

### Remove artists for which 'retrieved' value is 'none'

Convert none to null:

In [18]:
data['retrieved'] = data['retrieved'].replace('none', np.nan)

In [19]:
data.head()

,artist,gender,retrieved,genre
0,Billy Joe Royal,male,https://en.wikipedia.org/wiki/Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus..."
1,Inés Gaviria,female,https://en.wikipedia.org/wiki/Inés_Gaviria,"['Latin pop', 'Pop rock']"
2,Tha Realest,male,https://en.wikipedia.org/wiki/Tha_Realest,['Hip hop music']
3,Mr. Lif,male,https://en.wikipedia.org/wiki/Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli..."
4,Clarice Mayne,female,https://en.wikipedia.org/wiki/Clarice_Mayne,"['Music hall', 'Variety theatre']"


In [20]:
data.isnull().sum()

artist          0
gender          0
retrieved    7704
genre           0
dtype: int64

Drop rows with nulls:

In [21]:
data.dropna(axis = 0, inplace = True)

In [22]:
data.shape

(15473, 4)

In [23]:
data.head()

,artist,gender,retrieved,genre
0,Billy Joe Royal,male,https://en.wikipedia.org/wiki/Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus..."
1,Inés Gaviria,female,https://en.wikipedia.org/wiki/Inés_Gaviria,"['Latin pop', 'Pop rock']"
2,Tha Realest,male,https://en.wikipedia.org/wiki/Tha_Realest,['Hip hop music']
3,Mr. Lif,male,https://en.wikipedia.org/wiki/Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli..."
4,Clarice Mayne,female,https://en.wikipedia.org/wiki/Clarice_Mayne,"['Music hall', 'Variety theatre']"


## Remove the prefix from the 'retrieved' values

In [24]:
"""This function extracts artist name from the url.
Apply it to the 'retrieved' values."""
def retrieved_artist(text):
    try:
        retrieved = text
        p = re.compile(r'(https://en.wikipedia.org/wiki/)(.*)')
        result = re.match(p, retrieved)
        return result.group(2)
    except:
        if text == 'none':
            return 'none'
    else:
        return 'None'

In [25]:
data['retrieved'] = data.retrieved.apply(retrieved_artist)

In [26]:
data.head()

,artist,gender,retrieved,genre
0,Billy Joe Royal,male,Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus..."
1,Inés Gaviria,female,Inés_Gaviria,"['Latin pop', 'Pop rock']"
2,Tha Realest,male,Tha_Realest,['Hip hop music']
3,Mr. Lif,male,Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli..."
4,Clarice Mayne,female,Clarice_Mayne,"['Music hall', 'Variety theatre']"


## Replace spaces with _ in the artist column:

In [27]:
"""This function replaces white space in the values of
the column artist with an underscore."""
def underscore(text):
    try:
        split_name = text.split(' ')
        return '_'.join(split_name)  
    except:
        return 'error'

In [28]:
data['artist'] = data.artist.apply(underscore)

In [29]:
data.head()

,artist,gender,retrieved,genre
0,Billy_Joe_Royal,male,Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus..."
1,Inés_Gaviria,female,Inés_Gaviria,"['Latin pop', 'Pop rock']"
2,Tha_Realest,male,Tha_Realest,['Hip hop music']
3,Mr._Lif,male,Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli..."
4,Clarice_Mayne,female,Clarice_Mayne,"['Music hall', 'Variety theatre']"


## Remove the \_(singer) type designation from retrieved

In [30]:
"""This function uses re. to remove any parenthetical designations
form the retrieved artist name"""
def remove_designation(text):
    designations = [r'_\(singer\)', r'_\(musician\)', r'_\(rapper\)', r'_\(band\)', r'_\(composer\)', r'_\(music_producer\)']
    x = text
    for des in designations:
        if re.search(des, x):
            x = re.sub(r'{}'.format(des),'',text)
    return x

Apply the function:

In [31]:
data['retrieved_clean'] = data.retrieved.apply(remove_designation)

Take a glance at artist and retrieved_clean values:

In [32]:
rints = np.random.randint(0,data.shape[0],15) # generate 15 random numbers from 0 to k-1, with k = # of rows

data[['artist','retrieved_clean']].iloc[rints]

,artist,retrieved_clean
80,Frankie_Avalon,Frankie_Avalon
3756,Lily_Allen,Lily_Allen
18403,Lana_Lane,Lana_Lane
17969,Jeanne_Sagan,Jeanne_Sagan
7946,Nigel_Wright,Nigel_Wright
16949,Béla_Fleck,Béla_Fleck
19249,Ewen_Henderson,Ewen_Henderson
14680,Connie_Cato,Connie_Cato
17360,John_Dee_Holeman,John_Dee_Holeman
8560,Dave_Zegarac,Dave_Zegarac


### Mark the rows for which retrieved_clean is different from artist

In [33]:
"""This function takes a pair of strings and checks
if they are equivalent (case insensitive)

.casefold is used to be case insensitive; 
still might have problems on some characters"""

def verify_artist(x,y):
    if x.casefold() == y.casefold(): 
        return 1
    else:
        return 0

Introduce a mismatch just to make sure we can properly remove these:

In [34]:
# use an iloc index larger than the size of the original dataframe
#data.iloc[data.shape[0]+1] = ['test','test_wrong','universal','test_wrong']

Apply the function:

In [35]:
data['match'] = (data.artist.apply(lambda x: x.casefold()) != data.retrieved_clean.apply(lambda x: x.casefold())).astype('int64')

In [36]:
data.head()

,artist,gender,retrieved,genre,retrieved_clean,match
0,Billy_Joe_Royal,male,Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus...",Billy_Joe_Royal,0
1,Inés_Gaviria,female,Inés_Gaviria,"['Latin pop', 'Pop rock']",Inés_Gaviria,0
2,Tha_Realest,male,Tha_Realest,['Hip hop music'],Tha_Realest,0
3,Mr._Lif,male,Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli...",Mr._Lif,0
4,Clarice_Mayne,female,Clarice_Mayne,"['Music hall', 'Variety theatre']",Clarice_Mayne,0


In [37]:
data.match.sum()

0

Now remove artists where retrieved_clean doesn't match artist:

In [38]:
data = data[data.match == 0]

In [39]:
data.head()

,artist,gender,retrieved,genre,retrieved_clean,match
0,Billy_Joe_Royal,male,Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus...",Billy_Joe_Royal,0
1,Inés_Gaviria,female,Inés_Gaviria,"['Latin pop', 'Pop rock']",Inés_Gaviria,0
2,Tha_Realest,male,Tha_Realest,['Hip hop music'],Tha_Realest,0
3,Mr._Lif,male,Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli...",Mr._Lif,0
4,Clarice_Mayne,female,Clarice_Mayne,"['Music hall', 'Variety theatre']",Clarice_Mayne,0


Now the remaining artists are verified and have non-null genre label. 

### Genre Labels

Each value of the genre column is a _string_ of comma separated genre labels using the spotify abbreviations. We want to convert it to a _list_ of strings.

In [40]:
"""This function takes in a string of the form
appearing in the genrelist of the dataframe.
It strips the square brackets and extra quotes and
returns a list of strings where each string is a genre label."""
def genrelist(string):
    string = string.strip("[").strip("]").replace("'","")
    return [s for s in string.split(',')]

Now we apply it to the whole column and put the lists in a new column:

In [41]:
data['genrelist']= data['genre'].apply(genrelist)

In [42]:
data.head()

,artist,gender,retrieved,genre,retrieved_clean,match,genrelist
0,Billy_Joe_Royal,male,Billy_Joe_Royal,"['Rock and roll', 'Country music', 'Gospel mus...",Billy_Joe_Royal,0,"[Rock and roll, Country music, Gospel music]"
1,Inés_Gaviria,female,Inés_Gaviria,"['Latin pop', 'Pop rock']",Inés_Gaviria,0,"[Latin pop, Pop rock]"
2,Tha_Realest,male,Tha_Realest,['Hip hop music'],Tha_Realest,0,[Hip hop music]
3,Mr._Lif,male,Mr._Lif,"['Hip hop music', 'Alternative hip hop', 'Poli...",Mr._Lif,0,"[Hip hop music, Alternative hip hop, Politic..."
4,Clarice_Mayne,female,Clarice_Mayne,"['Music hall', 'Variety theatre']",Clarice_Mayne,0,"[Music hall, Variety theatre]"


### Remove all artists with null values for genre :

In [43]:
data = data[data['genrelist'].notnull()]

In [44]:
data.isnull().sum(axis = 0)

artist             0
gender             0
retrieved          0
genre              0
retrieved_clean    0
match              0
genrelist          0
dtype: int64

In [45]:
data.shape

(15473, 7)

Remove old columns:

In [46]:
data.columns

Index(['artist', 'gender', 'retrieved', 'genre', 'retrieved_clean', 'match',
       'genrelist'],
      dtype='object')

In [47]:
data.drop(['retrieved','genre','retrieved_clean', 'match'], axis = 1, inplace = True)

In [48]:
data.head()

,artist,gender,genrelist
0,Billy_Joe_Royal,male,"[Rock and roll, Country music, Gospel music]"
1,Inés_Gaviria,female,"[Latin pop, Pop rock]"
2,Tha_Realest,male,[Hip hop music]
3,Mr._Lif,male,"[Hip hop music, Alternative hip hop, Politic..."
4,Clarice_Mayne,female,"[Music hall, Variety theatre]"


In [49]:
data.shape

(15473, 3)

Remove any artists without gender value

In [50]:
data.gender.isnull().sum()

0

In [51]:
data.gender.unique()

array(['male', 'female'], dtype=object)

### Extracting the unique genre labels:

First make a list of the genrelists:

In [52]:
genre_list = data.genrelist.values.tolist()

In [53]:
genre_list[:5]

[['Rock and roll', ' Country music', ' Gospel music'],
 ['Latin pop', ' Pop rock'],
 ['Hip hop music'],
 ['Hip hop music', ' Alternative hip hop', ' Political hip hop'],
 ['Music hall', ' Variety theatre']]

Flatten:

In [54]:
genre_list = [x for y in genre_list for x in y]
len(genre_list)

39836

Only keep unique values:

In [55]:
genre_list = list(set(genre_list))

In [56]:
len(genre_list)

2440

In [57]:
print('There are {} artists with genre and binary-gender labels.'.format(data.shape[0]))
print('There are {} unique genre labels.'.format(len(genre_list)))

There are 15473 artists with genre and binary-gender labels.
There are 2440 unique genre labels.


## Select 10% of artists to verify gender manually.

In [58]:
data.shape

(15473, 3)

In [59]:
data_male = data[data.gender == 'male']
data_female = data[data.gender == 'female']

In [72]:
tot = data.shape[0]
m = data_male.shape[0]
f = data_female.shape[0]
print('{} total artists'.format(tot))
print('{} female artists, or {:0.0f}%'.format(f, 100*f/(f+m)))
print('{} male artists, or {:0.0f}%'.format(m, 100*m/(f+m)))

15473 total artists
4864 female artists, or 31%
10609 male artists, or 69%


We will verify the same number of female as male artists. Let 
- $p$ be the fraction of artists to verify,
- $p_f$ be the fraction of female artists to verify
- $p_m$ be the fraction of male artists to verify

We want two conditions satisfied:

$$ p_f \cdot f = p_m \cdot m $$
$$ p_f \cdot f + p_m \cdot m = p \cdot (f+m)$$

These encode the requirements that
- we verify equal numbers of male and female artists
- the fractino of artists we verify is $p$

Solving for $p_f$ and $p_m$ leads to 

$$ p_f = \frac{p}{2f}(f+m)$$
$$ p_m = \frac{p}{2m}(f+m)$$

Substituting in $p=.1$, $f = 4864$, and $m = 10609$ leads to 

In [74]:
def pf(p,f,m):
    return p*(m+f)/(2*f)

def pm(p,f,m):
    return p*(m+f)/(2*m)

The values for our data are:

In [89]:
pf(.1,f,m), pm(.1,f,m)

(0.15905633223684212, 0.07292393251013292)

We select samples from the data:

In [90]:
sample_male = data_male.sample(frac = .0729)
sample_female = data_female.sample(frac = .159)

In [91]:
sample_male.shape[0], sample_female.shape[0]

(773, 773)